In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# Parameters
data_dir = '/home/sivaranjin/lost+found/Speech/Urban Scene/Database/Database'  # Replace with your data directory
batch_size = 64
img_height, img_width = 224, 224
num_classes = 4
learning_rate = 0.0001
num_epochs = 100

In [ ]:
# Data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Load the data from directory
train_generator = train_datagen.flow_from_directory(
    directory=f'{data_dir}/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    class_mode='categorical'
)


val_generator = val_datagen.flow_from_directory(
    f'{data_dir}/val',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Compute class weights for imbalanced dataset
y_train = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

print("Class weights:", class_weights)


In [ ]:
# #Model 01

# import tensorflow as tf
# from tensorflow.keras.layers import (
#     Conv2D, DepthwiseConv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, 
#     Dense, Dropout, BatchNormalization, ReLU, Input
# )
# from tensorflow.keras.models import Model

# def create_custom_cnn(input_shape=(224, 224, 3), num_classes=4):
#     inputs = Input(shape=input_shape)
    
#     # Convolutional Block 1
#     x = Conv2D(64, (3, 3), padding="same", activation=None)(inputs)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
#     x = MaxPooling2D((2, 2))(x)
    
#     # Convolutional Block 2
#     x = Conv2D(128, (3, 3), padding="same", activation=None)(x)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
#     x = MaxPooling2D((2, 2))(x)
    
#     # Convolutional Block 3
#     x = SeparableConv2D(256, (3, 3), padding="same", activation=None)(x)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
#     x = MaxPooling2D((2, 2))(x)
    
#     # Convolutional Block 4
#     x = SeparableConv2D(512, (3, 3), padding="same", activation=None)(x)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
#     x = GlobalAveragePooling2D()(x)
    
#     # Dense Layer
#     x = Dense(256, activation="relu")(x)
#     x = Dropout(0.4)(x)
    
#     # Output Layer
#     outputs = Dense(num_classes, activation="softmax")(x)
    
#     # Model definition
#     model = Model(inputs, outputs)
#     return model

# # Instantiate the model
# model = create_custom_cnn()

# # Compile the model
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#     loss="categorical_crossentropy",
#     metrics=["accuracy"]
# )

In [ ]:
# #Model 02
# import tensorflow as tf
# from tensorflow.keras.layers import (
#     Conv2D, DepthwiseConv2D, SeparableConv2D, Dense, BatchNormalization, 
#     ReLU, GlobalAveragePooling2D, Add, Dropout, Input, AveragePooling2D
# )
# from tensorflow.keras.models import Model

# def squeeze_and_excite_block(inputs, ratio=16):
#     filters = inputs.shape[-1]
#     se = GlobalAveragePooling2D()(inputs)
#     se = Dense(filters // ratio, activation="relu")(se)
#     se = Dense(filters, activation="sigmoid")(se)
#     return tf.keras.layers.Multiply()([inputs, se])

# def dense_residual_block(x, growth_rate, use_se=False):
#     # Bottleneck layer
#     conv1 = Conv2D(growth_rate, (1, 1), padding="same", activation=None)(x)
#     conv1 = BatchNormalization()(conv1)
#     conv1 = ReLU()(conv1)
    
#     # Depthwise separable convolution
#     conv2 = SeparableConv2D(growth_rate, (3, 3), padding="same", activation=None)(conv1)
#     conv2 = BatchNormalization()(conv2)
#     conv2 = ReLU()(conv2)
    
#     # Attention block
#     if use_se:
#         conv2 = squeeze_and_excite_block(conv2)
    
#     # Concatenate input and output for dense connections
#     return tf.keras.layers.Concatenate()([x, conv2])

# def create_hybrid_cnn(input_shape=(224, 224, 3), num_classes=4):
#     inputs = Input(shape=input_shape)
    
#     # Stem Block
#     x = SeparableConv2D(32, (3, 3), strides=(2, 2), padding="same")(inputs)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
    
#     # Dense Residual Blocks
#     for _ in range(3):  # Add 3 dense residual blocks
#         x = dense_residual_block(x, growth_rate=64, use_se=True)
#         x = AveragePooling2D((2, 2))(x)  # Transition layer
    
#     # Global Average Pooling
#     x = GlobalAveragePooling2D()(x)
    
#     # Fully Connected Layer
#     x = Dense(256, activation="relu")(x)
#     x = Dropout(0.4)(x)
    
#     # Output Layer
#     outputs = Dense(num_classes, activation="softmax")(x)
    
#     # Create Model
#     model = Model(inputs, outputs)
#     return model

# # Instantiate the model
# model = create_hybrid_cnn()

# # Compile the model
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#     loss="categorical_crossentropy",
#     metrics=["accuracy"]
# )

In [ ]:
# #Model 03

# import tensorflow as tf
# from tensorflow.keras.layers import (
#     Input, Conv2D, DepthwiseConv2D, SeparableConv2D, BatchNormalization, ReLU, 
#     Add, Concatenate, GlobalAveragePooling2D, Dense, Dropout, AveragePooling2D
# )
# from tensorflow.keras.models import Model

# def squeeze_and_excite(inputs, reduction_ratio=16):
#     """Squeeze-and-Excitation block for lightweight attention."""
#     filters = inputs.shape[-1]
#     se = GlobalAveragePooling2D()(inputs)
#     se = Dense(filters // reduction_ratio, activation="relu")(se)
#     se = Dense(filters, activation="sigmoid")(se)
#     return tf.keras.layers.Multiply()([inputs, se])

# def dense_residual_block(inputs, filters, use_attention=True):
#     """Dense Residual Block combining DenseNet and ResNet features."""
#     # Bottleneck Convolution
#     x = BatchNormalization()(inputs)
#     x = ReLU()(x)
#     x = Conv2D(filters, (1, 1), padding="same", use_bias=False)(x)

#     # Depthwise Separable Convolution
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
#     x = SeparableConv2D(filters, (3, 3), padding="same", use_bias=False)(x)
    
#     # Attention Mechanism
#     if use_attention:
#         x = squeeze_and_excite(x)
    
#     # Align input shape with output shape
#     if inputs.shape[-1] != filters:
#         inputs = Conv2D(filters, (1, 1), padding="same", use_bias=False)(inputs)
    
#     # Add residual connection
#     return Add()([inputs, x])


# def create_hybrid_cnn(input_shape=(224, 224, 3), num_classes=4):
#     inputs = Input(shape=input_shape)

#     # Stem Block: Initial feature extraction
#     x = Conv2D(32, (3, 3), strides=(2, 2), padding="same", use_bias=False)(inputs)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
    
#     x = DepthwiseConv2D((3, 3), strides=(1, 1), padding="same", use_bias=False)(x)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
    
#     # Dense Residual Blocks
#     for filters in [64, 128, 256]:  # Incrementing filters
#         x = dense_residual_block(x, filters)
#         x = AveragePooling2D((2, 2))(x)  # Transition layer

#     # Final Convolution Block
#     x = Conv2D(512, (1, 1), padding="same", use_bias=False)(x)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
    
#     # Global Average Pooling
#     x = GlobalAveragePooling2D()(x)

#     # Fully Connected Layer
#     x = Dense(256, activation="relu")(x)
#     x = Dropout(0.4)(x)

#     # Output Layer
#     outputs = Dense(num_classes, activation="softmax")(x)

#     # Create Model
#     model = Model(inputs, outputs)
#     return model

# # Instantiate the model
# model = create_hybrid_cnn()

# # Compile the model
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#     loss="categorical_crossentropy",
#     metrics=["accuracy"]
# )

In [ ]:
# #Model 04

# import tensorflow as tf
# from tensorflow.keras.layers import (
#     Input, Conv2D, DepthwiseConv2D, SeparableConv2D, BatchNormalization, ReLU, 
#     Add, GlobalAveragePooling2D, Dense, Dropout, AveragePooling2D, Concatenate
# )
# from tensorflow.keras.models import Model

# def squeeze_and_excite(inputs, reduction_ratio=16):
#     """Squeeze-and-Excitation block."""
#     filters = inputs.shape[-1]
#     se = tf.keras.layers.GlobalAveragePooling2D()(inputs)
#     se = Dense(filters // reduction_ratio, activation="relu")(se)
#     se = Dense(filters, activation="sigmoid")(se)
#     return tf.keras.layers.Multiply()([inputs, se])

# def dense_residual_block(inputs, filters, use_attention=True):
#     """Dense Residual Block with attention."""
#     x = BatchNormalization()(inputs)
#     x = ReLU()(x)
#     x = Conv2D(filters, (1, 1), padding="same", use_bias=False)(x)

#     x = BatchNormalization()(x)
#     x = ReLU()(x)
#     x = SeparableConv2D(filters, (3, 3), padding="same", use_bias=False)(x)
    
#     if use_attention:
#         x = squeeze_and_excite(x)

#     if inputs.shape[-1] != filters:
#         inputs = Conv2D(filters, (1, 1), padding="same", use_bias=False)(inputs)
    
#     return Add()([inputs, x])

# def multi_scale_feature_fusion(inputs):
#     """Fuse features at multiple scales."""
#     x1 = Conv2D(inputs.shape[-1] // 2, (1, 1), padding="same")(inputs)
#     x2 = Conv2D(inputs.shape[-1] // 2, (3, 3), padding="same")(inputs)
#     x3 = Conv2D(inputs.shape[-1] // 2, (5, 5), padding="same")(inputs)
#     return Concatenate()([x1, x2, x3])

# def create_hybrid_cnn(input_shape=(224, 224, 3), num_classes=4):
#     inputs = Input(shape=input_shape)

#     # Stem Block
#     x = Conv2D(32, (3, 3), strides=(2, 2), padding="same", use_bias=False)(inputs)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)

#     # Multi-Scale Feature Fusion
#     x = multi_scale_feature_fusion(x)

#     # Dense Residual Blocks
#     for filters in [64, 128, 256]:
#         x = dense_residual_block(x, filters)
#         x = AveragePooling2D((2, 2))(x)

#     # Final Convolution Block
#     x = Conv2D(512, (1, 1), padding="same", use_bias=False)(x)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)

#     # Global Average Pooling
#     x = GlobalAveragePooling2D()(x)

#     # Fully Connected Layers
#     x = Dense(256, activation="relu")(x)
#     x = Dropout(0.4)(x)
#     outputs = Dense(num_classes, activation="softmax")(x)

#     model = Model(inputs, outputs)
#     return model

# # Instantiate the model
# model = create_hybrid_cnn()

# # Compile the model
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#     loss="categorical_crossentropy",
#     metrics=["accuracy"]
# )

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, 
    Dropout, BatchNormalization, ReLU, Input, Add
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

def squeeze_and_excite(x, ratio=16):
    """Squeeze-and-Excite block to recalibrate the feature maps."""
    channel_axis = -1
    filters = x.shape[channel_axis]
    
    # Squeeze
    se = GlobalAveragePooling2D()(x)
    se = Dense(filters // ratio, activation="relu", use_bias=False)(se)
    se = Dense(filters, activation="sigmoid", use_bias=False)(se)
    
    # Scale
    se = tf.reshape(se, [-1, 1, 1, filters])
    return x * se

def residual_block(x, filters, kernel_size=(3, 3), strides=(1, 1), use_se=False):
    """Residual block with an optional squeeze and excite mechanism."""
    shortcut = x
    
    # Apply 1x1 convolution to shortcut to match the output shape
    if x.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), padding="same", strides=strides, activation=None)(x)
        shortcut = BatchNormalization()(shortcut)
    
    # Convolutional Layers
    x = Conv2D(filters, kernel_size, strides=strides, padding="same", activation=None)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Optional Squeeze-and-Excite block
    if use_se:
        x = squeeze_and_excite(x)
    
    x = Conv2D(filters, kernel_size, strides=strides, padding="same", activation=None)(x)
    x = BatchNormalization()(x)
    
    # Add shortcut
    x = Add()([x, shortcut])
    x = ReLU()(x)
    
    return x

def create_custom_cnn(input_shape=(224, 224, 3), num_classes=4, learning_rate=1e-4):
    inputs = Input(shape=input_shape)
    
    # Initial Conv Block
    x = Conv2D(64, (3, 3), padding="same", activation=None)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Residual Block 1
    x = residual_block(x, 128, use_se=True)
    
    # Residual Block 2
    x = residual_block(x, 256, use_se=True)
    
    # Separable Conv Block (to reduce the number of parameters)
    x = SeparableConv2D(512, (3, 3), padding="same", activation=None)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)
    
    # Dense Layer with L2 Regularization
    x = Dense(256, activation="relu", kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.4)(x)
    
    # Output Layer
    outputs = Dense(num_classes, activation="softmax")(x)
    
    # Model definition
    model = Model(inputs, outputs)
    return model

# Instantiate the model
model = create_custom_cnn()

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "custom_model7.h5", save_best_only=True, monitor="val_loss"
    ),
tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.1, patience=7, min_lr=1e-6
    ),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, verbose=1, restore_best_weights=True),
]

In [ ]:
# Display model summary to get parameter details
model.summary()

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    class_weight=class_weights,
    verbose=1,
    callbacks = callbacks
)

In [ ]:
# Plot accuracy and loss curves
def plot_training_curves(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(num_epochs)
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Train')
    plt.plot(val_acc, label='Validation')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Train')
    plt.plot(val_loss, label='Validation')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

plot_training_curves(history)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# Confusion Matrix
y_true = val_generator.classes
y_pred = np.argmax(model.predict(val_generator), axis=1)

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_generator.class_indices.keys())
disp.plot(cmap='Blues')
plt.show()

In [ ]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy*100:.2f}%")
# print(f"Validation Precision: {precision:.2f}")
# print(f"Validation Recall: {recall:.2f}")

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
from pytictoc import TicToc

In [ ]:
# IMPORTANT
# run this cell minimum 2 times to get approximate correct-timing results.

t = TicToc()
t.tic()
y_pred = np.argmax(model.predict(val_generator), axis=1)
t.toc('\nThis testing took')
total_time = t.tocvalue('This testing took')
total_seg = y_pred.shape[0]

print("Average testing time is {} milliseconds\n".format(total_time*1000/total_seg))

In [ ]:
from tensorflow.keras.models import Model

feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)
features = feature_extractor.predict(val_generator)
labels = val_generator.classes

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=3, random_state=0)
tsne_results = tsne.fit_transform(features)

In [ ]:
tsne_results.shape

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame(tsne_results, columns=['t-SNE x1', 't-SNE x2', 't-SNE x3'])
df['Classes'] = labels

fig = px.scatter_3d(df, x='t-SNE x1', y='t-SNE x2', z='t-SNE x3', color='Classes', size_max=5)
fig.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Create scatter plot
scatter = ax.scatter(tsne_results[:, 0], tsne_results[:, 1], tsne_results[:, 2], c=labels, cmap='plasma', marker='o', s = 2)

# # Add legend
# legend1 = ax.legend(['cry', 'non-cry'], loc="upper right",  title="Classes")
# ax.add_artist(legend1)

# Custom legend
class_names = {0: 'Cycle', 1: 'Motorcycle', 2: 'Pedestrians', 3: 'Traffic'}
handles, _ = scatter.legend_elements()
legend_labels = [class_names[int(label)] for label in np.unique(labels)]
legend = ax.legend(handles, legend_labels, loc="upper right")

# Set labels
ax.set_xlabel('t-SNE x1')
ax.set_ylabel('t-SNE x2')
ax.set_zlabel('t-SNE x3')

# Show plot
plt.show()


In [ ]:
fig.savefig("custom_model7_t-SNE.pdf") 